In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nlppln
import ochre
import os

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    wf.load(step_file='https://raw.githubusercontent.com/nlppln/ocrevaluation-docker/master/ocrevaluation.cwl')
    
    print wf.list_steps()

    gt_dir = wf.add_input(gt='Directory')
    ocr_dir = wf.add_input(ocr='Directory')
    performance_file = wf.add_input(out_name='string?', default='performance.csv')

    ocr_files = wf.ls(in_dir=ocr_dir)
    gt_files = wf.ls(in_dir=gt_dir)
    out_files = wf.ocrevaluation(gt=gt_files, ocr=ocr_files, scatter=['gt', 'ocr'], scatter_method='dotproduct')
    character_data, global_data = wf.ocrevaluation_extract(in_file=out_files, scatter=['in_file'], 
                                                           scatter_method='dotproduct')
    merged = wf.merge_csv(in_files=global_data, name=performance_file)

    wf.add_outputs(performance=merged)

    # save multiple versions - packed (can't be used as subworkflow)
    wf.save(os.path.join(ochre.cwl_path(), 'ocrevaluation-performance-wf-pack.cwl'), pack=True)
    # can be used as a subworkflow
    wf.save(os.path.join(ochre.cwl_path(), 'ocrevaluation-performance-wf.cwl'), wd=True, relative=False)

In [ ]:
import nlppln

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    wf.load(step_file='https://raw.githubusercontent.com/nlppln/ocrevaluation-docker/master/ocrevaluation.cwl')
    print(wf.list_steps())

    ocr_dir = wf.add_input(ocr='Directory')
    gt_dir = wf.add_input(gt='Directory')
    datadiv = wf.add_input(datadivision='File')
    gt_dir_name = wf.add_input(gt_dir_name='string', default='gs')
    ocr_dir_name = wf.add_input(ocr_dir_name='string', default='ocr')
    fname = wf.add_input(out_name='string?', default='performance.csv')

    ocr_files = wf.select_test_files(datadivision=datadiv, in_dir=ocr_dir)
    gt_files = wf.select_test_files(datadivision=datadiv, in_dir=gt_dir)

    gt_dir = wf.save_files_to_dir(dir_name=gt_dir_name, in_files=gt_files)
    ocr_dir = wf.save_files_to_dir(dir_name=ocr_dir_name, in_files=ocr_files)

    merged = wf.ocrevaluation_performance_wf(gt=gt_dir, ocr=ocr_dir, fname=fname)

    wf.add_outputs(performance=merged)

    wf.save(os.path.join(ochre.cwl_path(), 'ocrevaluation-performance-test-files-wf-pack.cwl'), pack=True, relative=False)